In [6]:
import torch
torch.cuda.is_available()

False

In [7]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [8]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm

In [9]:
print("System Version:", sys.version)
print("PyTorch version", torch.__version__)
print("Torchvision version", torchvision.__version__)
print("Numpy version", np.__version__)
print("Pandas version", pd.__version__)

System Version: 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]
PyTorch version 2.5.1+cu124
Torchvision version 0.20.1+cu124
Numpy version 2.2.1
Pandas version 2.2.3


In [10]:
class PlayingCardDataset(Dataset):
    def __init__(self, data_dir, transform = None):
        self.data = ImageFolder(data_dir, transform= transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
    
    @property
    def classes(self):
        return self.data.classes

In [11]:
dataset = PlayingCardDataset(data_dir='/home/om/Downloads/archive_1/train')

In [12]:
len(dataset)

7624

In [13]:
image, label = dataset[6000]
# print(label)
image.size

(224, 224)

In [14]:
dataset[200]

(<PIL.Image.Image image mode=RGB size=224x224>, 1)

In [15]:
data_dir = '/home/om/Downloads/archive_1/train'
target_to_class = {v: k for k,v in ImageFolder(data_dir).class_to_idx.items()}
print(target_to_class)

{0: 'ace of clubs', 1: 'ace of diamonds', 2: 'ace of hearts', 3: 'ace of spades', 4: 'eight of clubs', 5: 'eight of diamonds', 6: 'eight of hearts', 7: 'eight of spades', 8: 'five of clubs', 9: 'five of diamonds', 10: 'five of hearts', 11: 'five of spades', 12: 'four of clubs', 13: 'four of diamonds', 14: 'four of hearts', 15: 'four of spades', 16: 'jack of clubs', 17: 'jack of diamonds', 18: 'jack of hearts', 19: 'jack of spades', 20: 'joker', 21: 'king of clubs', 22: 'king of diamonds', 23: 'king of hearts', 24: 'king of spades', 25: 'nine of clubs', 26: 'nine of diamonds', 27: 'nine of hearts', 28: 'nine of spades', 29: 'queen of clubs', 30: 'queen of diamonds', 31: 'queen of hearts', 32: 'queen of spades', 33: 'seven of clubs', 34: 'seven of diamonds', 35: 'seven of hearts', 36: 'seven of spades', 37: 'six of clubs', 38: 'six of diamonds', 39: 'six of hearts', 40: 'six of spades', 41: 'ten of clubs', 42: 'ten of diamonds', 43: 'ten of hearts', 44: 'ten of spades', 45: 'three of clu

In [16]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
])

data_dir = '/home/om/Downloads/archive_1/train'
dataset = PlayingCardDataset(data_dir, transform)

In [18]:
image, label = dataset[100]
image.shape

torch.Size([3, 128, 128])

In [23]:
for image, label in dataset:
    print(image.shape)
    print(label)
    break
    

torch.Size([3, 128, 128])
0


# Dataloaders

In [41]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [42]:
for images, labels in dataloader:
    print(image.shape)
    print(label.shape)
    break

torch.Size([3, 128, 128])
torch.Size([32])


In [45]:
images.shape, labels.shape

(torch.Size([32, 3, 128, 128]), torch.Size([32]))

In [44]:
labels

tensor([12, 17, 29, 38, 14,  5, 38,  8, 35, 33,  8, 33, 14,  4, 15,  7, 30, 12,
        24, 33, 31,  5,  0, 28,  0, 19, 12, 25, 32, 29, 14, 28])

# Pytorch Model

In [30]:
class SimpleCardClassifer(nn.Module):
    def __init__(self, num_classes=53):
        super(SimpleCardClassifer, self).__init__()

        self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])

        enet_out_size = 1280

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(enet_out_size, num_classes)
        )
    
    def forward(self, x):

        x=self.features(x)
        output = self.classifier(x)
        return output

In [ ]:
model = SimpleCardClassifer(num_classes=53)
#printing the last few characters of the model
print(str(model)[-500:])

Identity()
        )
      )
    )
    (3): Conv2d(320, 1280, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (4): BatchNormAct2d(
      1280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (5): SelectAdaptivePool2d(pool_type=avg, flatten=Flatten(start_dim=1, end_dim=-1))
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1280, out_features=53, bias=True)
  )
)


In [39]:
example_out = model(images)
example_out.shape

torch.Size([32, 53])

# The training loop

In [40]:
#Loss function
criterion = nn.CrossEntropyLoss()
#Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [46]:
criterion(example_out, labels)

tensor(4.0155, grad_fn=<NllLossBackward0>)

In [47]:
print(example_out.shape)

torch.Size([32, 53])


In [48]:
print(labels.shape)

torch.Size([32])


## Setup datasets

In [49]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
])

train_folder = '/home/om/Downloads/archive_1/train'
valid_folder = '/home/om/Downloads/archive_1/valid'
test_folder = '/home/om/Downloads/archive_1/test'

train_dataset = PlayingCardDataset(train_folder, transform=transform) 
val_dataset = PlayingCardDataset(valid_folder, transform=transform)
test_dataset = PlayingCardDataset(test_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

## Simple Training Loop

In [ ]:
num_epochs = 5
train_losses, val_losses = [], []

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = SimpleCardClassifer(num_classes= 53)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    #training phase
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc='Trainingloop'):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)
    train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(train_loss)
        
